**Add a Calibration Measure to the OpenStudio Workflow (OSW)**

The OpenStudio SDK provides a tool for programmatically making changes to an OpenStudio Model. By using the OpenStudio Workflow (OSW), you can define a series of measures that make these changes in a specified order. We will add a calibration measure from the calibration-measures gem to the baseline model workflow described in BaselineModel.ipynb. We will also change measure arguments by modifying the measure's properties in the OSW file.

To begin, load the 'openstudio' Ruby bindings to the SDK as well as 'fileutils' for basic file/directory manipulation.

In [1]:
require 'fileutils'
require 'openstudio'

true

create project directory called 'Calibration' and change the working directory to match.

In [2]:
project_dir = 'Calibration'
#create project folder
FileUtils.mkdir_p("#{project_dir}") unless Dir.exist?("#{project_dir}")

#change working directory
Dir.chdir "#{project_dir}"

0

Create the example model and save in the 'seeds' folder

In [3]:
# Create the Example Model
model = OpenStudio::Model.exampleModel()

#create directory for the seed model
FileUtils.mkdir_p("seeds") unless Dir.exist?("seeds")
# Save the model to an OSM file
osm_path = OpenStudio::Path.new(File.expand_path("seeds/example_model.osm"))
model.save(osm_path, true)


true

Get the weather file from previous download or redownload and put in the weather directory.  Set the weather file into the Model and do some basic validation on it. 

In [4]:
#get weather file
require 'open-uri'

# URL of the file to download
url = 'https://github.com/NREL/OpenStudio/raw/develop/resources/utilities/Filetypes/'

# Local file name
weather_file_name = 'USA_CO_Golden-NREL.724666_TMY3.epw'
weather_file_path = "weather/" + weather_file_name

#create directory for the weather file
FileUtils.mkdir_p("weather") unless Dir.exist?("weather")

if !File.exist?(weather_file_path)
    puts "Download the file #{weather_file_name}"
    URI.open(url + weather_file_name) do |file|
      File.write(weather_file_path, file.read)
    end
else
    puts 'weather file exists locally'
end

#set the weather file in the OSM
#use absolute path
epw_file = OpenStudio::EpwFile.load(OpenStudio::Path.new(File.expand_path(weather_file_path)), true)
OpenStudio::Model::WeatherFile::setWeatherFile(model, epw_file.get)
weatherFile = model.getWeatherFile
# validate the weather file
puts weatherFile.checksum.get == 'BDF687C1'
puts weatherFile.country == 'USA'
puts weatherFile.stateProvinceRegion == 'CO'

Download the file USA_CO_Golden-NREL.724666_TMY3.epw
true
true
true


use the 'Open3' library to make a system call to the OpenStudio command-line interface (CLI) and check its version. The CLI path is stored in the 'cli' variable and the system call is made using the 'capture3' method of the 'Open3' module, which captures the standard output, standard error, and exit status of the system command. The result of the system call is checked using the 'status.success?' method, and if it is successful, the standard output is printed to the console. If it fails, the standard error is printed instead.

In [5]:
require 'open3'

#get path to OpenStudio CLI
cli = OpenStudio.getOpenStudioCLI.to_s

# Make a system call to get version
stdout, stderr, status = Open3.capture3("#{cli} --version")

# Check the result
if status.success?
  puts "Command executed successfully"
  puts stdout
else
  puts "Command failed"
  puts stderr
end

Command executed successfully
3.5.0+7b14ce1588


require the "openstudio-calibration" and "openstudio-common-measures" ruby gems. These gems contain pre-written scripts (OpenStudio Measures) that can make programmatic changes to an OpenStudio model, that we want to use in our project. Use the "Gem::Specification" method or the "Gem.find_files" method to find the local path to the /lib folder of the gem which is where the Measures are located. These paths are then stored in the variables "calibration_measures_dir" and "common_measures_dir".

In [6]:
require 'openstudio-calibration'
require 'openstudio-common-measures'

#find path to the Measures
puts Pathname.new(Gem.find_files('openstudio-common-measures.rb').first).dirname
# -or
#find path to Measures using gem::spec
common_measures_dir = Gem::Specification.find_by_name("openstudio-common-measures").gem_dir
calibration_measures_dir = Gem::Specification.find_by_name("openstudio-calibration").gem_dir

C:/Ruby27-x64/lib/ruby/gems/2.7.0/gems/openstudio-common-measures-0.7.0/lib


"C:/Ruby27-x64/lib/ruby/gems/2.7.0/gems/openstudio-calibration-0.7.0"

copy the OpenStudio Measures from the openstudio-common-measures and openstudio-calibration gems into a local project directory called "measures". The purpose of this is to make these scripts available for use in the current project. The measures copied include "openstudio_results", "AddMonthlyJSONUtilityData", "CalibrationReportsEnhanced", and "GeneralCalibrationMeasurePercentChange".

In [7]:
#make measure directories for the project

FileUtils.mkdir_p("measures") unless Dir.exist?("measures")

#copy measure from gem to local project directory
FileUtils.cp_r("#{common_measures_dir}/lib/measures/openstudio_results", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/AddMonthlyJSONUtilityData", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/CalibrationReportsEnhanced", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/GeneralCalibrationMeasurePercentChange", "measures")

create a project directory named "data" if it doesn't already exist, and then copies the monthly metered data in the "Data" directory located outside of the project folder to the newly created "data" folder within the project folder. The data that is being copied is in JSON format.

The OSW file assumes that the associated files and directories are in the same location.

  /calibration_workflow.osw  
  /measures  
  /seeds  
  /weather   
  /data  



In [8]:
#make directory for the calibration data
FileUtils.mkdir_p("data") unless Dir.exist?("data")

#copy JSON data to local project directory
FileUtils.cp_r("../Data/.", "data")

Create the OSW as before and add the GeneralCalibrationMeasurePercentChange to the ModelMeasure step. This is a general purpose measure that can be used for changing of space and spaceType loads as well as infiltration, and outdoor air with a percent change. The user can choose between a SINGLE SpaceType or ALL the SpaceTypes as well as a SINGLE Space or ALL the Spaces.

Change the Space Lights, Electric Equipment, People, Mass, Infiltration and Ventilation all by 10% and add to the OSW.

In [9]:
# Create a new OSW
osw = OpenStudio::WorkflowJSON.new

# Define the reporting measure steps
measure_steps = []

#add Utility Bill object to model from external JSON file
os_results = OpenStudio::MeasureStep.new("measures/AddMonthlyJSONUtilityData")
os_results.setArgument("json","../../../data/electric.json")
os_results.setArgument("variable_name","Electricity Bill")
os_results.setArgument("fuel_type","Electricity")
os_results.setArgument("consumption_unit","kWh")
os_results.setArgument("data_key_name","tot_kwh")
os_results.setArgument("start_date","2013-01-1")
os_results.setArgument("end_date","2013-12-31")
os_results.setArgument("remove_existing_data",TRUE)
os_results.setArgument("set_runperiod",TRUE)
measure_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/AddMonthlyJSONUtilityData")
os_results.setArgument("json","../../../data/natural_gas.json")
os_results.setArgument("variable_name","Gas Bill")
os_results.setArgument("fuel_type","Gas")
os_results.setArgument("consumption_unit","therms")
os_results.setArgument("data_key_name","tot_therms")
os_results.setArgument("start_date","2013-01-1")
os_results.setArgument("end_date","2013-12-31")
os_results.setArgument("remove_existing_data",FALSE)
#os_results.setArgument("set_runperiod",TRUE)
measure_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/GeneralCalibrationMeasurePercentChange")
os_results.setArgument("space_type","*All SpaceTypes*")
os_results.setArgument("space","*All Spaces*")
os_results.setArgument("lights_perc_change",10.0)
os_results.setArgument("ElectricEquipment_perc_change",10.0)
os_results.setArgument("people_perc_change",10.0)
os_results.setArgument("mass_perc_change",10.0)
os_results.setArgument("infil_perc_change",10.0)
os_results.setArgument("vent_perc_change",10.0)
measure_steps << os_results

# Set the measure steps in the OSW
measure_type = OpenStudio::MeasureType.new("ModelMeasure")
osw.setMeasureSteps(measure_type, measure_steps)

# Define the reporting measure steps
reporting_steps = []

#add openstudio_results to the reporting_steps array
os_results = OpenStudio::MeasureStep.new("measures/openstudio_results")
reporting_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/CalibrationReportsEnhanced")
reporting_steps << os_results

# Set the measure steps in the OSW
measure_type = OpenStudio::MeasureType.new("ReportingMeasure")
osw.setMeasureSteps(measure_type, reporting_steps)

true

Add the example_model OSM we generated earlier as the 'seed' model and set the weatherfile in the OSW. Also, save the OSW as 'baseline_model.osw' by creating a full, absolute path to the file name and use the .saveAs() method.

In [10]:
#add seed / weatherfile to OSW
osw.setSeedFile(osm_path)
osw.setWeatherFile(weather_file_path)
#save file locally
osw_path = OpenStudio::Path.new(File.expand_path("calibration_workflow.osw"))
osw.saveAs(osw_path)

true

Since this is in Ruby and the CLI is an executable, to run the workflow using the CLI, we will use the Open3.capture3 method to make a System Call to the CLI, which captures the standard output, standard error, and status code of the CLI command. The CLI command runs the workflow, specifying the workflow file path, and the --debug and --workflow options are passed to the CLI. Finally, the status code of the CLI command is stored in the status variable.

In [11]:
#Run the workflow
stdout, stderr, status = Open3.capture3("#{cli} run --debug --workflow #{osw_path}")
status

#<Process::Status: pid 17708 exit 0>

To view the results, set the file path to an HTML report generated by the "CalibrationReportsEnhanced" measure and display the contents of the file using IRuby's display method. The argument passed to the display method is the content of the file read using Ruby's File.read method, and the mime type is specified as 'text/html' to indicate that the contents are HTML. This code assumes that the file exists and will raise an error if it cannot be found or read.

In [14]:
file_path = File.expand_path(Dir.pwd + '/run/004_measures/CalibrationReportsEnhanced/report.html')

IRuby.display File.read(file_path), mime: 'text/html'

"<!DOCTYPE html>\n<html>\n<head>\n<meta charset=\"utf-8\">\n<title>Calibration | OpenStudio</title>\n<link href=\"http://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/2.3.2/css/bootstrap.min.css\" rel=\"stylesheet\">\n<style type=\"text/css\">\nbody {\n    font: 10px sans-serif;\n    min-width: 750px;\n}\ntable {\n    max-width:700px;\n}\n.axis path, .axis line {\n    fill: none;\n    stroke: #000;\n    shape-rendering: crispEdges;\n}\n.bar {\n    fill: steelblue;\n}\n.x.axis path {\n    display: none;\n}\n.d3-tip {\n    line-height: 1;\n    font-weight: bold;\n    padding: 12px;\n    background: rgba(0, 0, 0, 0.8);\n    color: #fff;\n    border-radius: 2px;\n}\n/* Creates a small triangle extender for the tooltip */\n.d3-tip:after {\n    box-sizing: border-box;\n    display: inline;\n    font-size: 10px;\n    width: 100%;\n    line-height: 1;\n    color: rgba(0, 0, 0, 0.8);\n    content:\"\\25BC\";\n    position: absolute;\n    text-align: center;\n}\n/* Style northward tooltips differently */\n.d3-tip.n:after {\n    margin: -1px 0 0 0;\n    top: 100%;\n    left: 0;\n}\n.chartInfo h6 {\n    display: inline;\n}\n</style>\n<script type=\"text/javascript\" src=\"http://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js\"></script>\n<script type=\"text/javascript\" src=\"http://cdnjs.cloudflare.com/ajax/libs/d3/3.3.9/d3.min.js\"></script>\n<script type=\"text/javascript\">\n// http://labratrevenge.com/d3-tip/javascripts/d3.tip.min.js\nd3.tip=function(){function t(t){v=m(t),w=v.createSVGPoint(),document.body.appendChild(x)}function e(){return\"n\"}function n(){return[0,0]}function r(){return\" \"}function o(){var t=p();return{top:t.n.y-x.offsetHeight,left:t.n.x-x.offsetWidth/2}}function s(){var t=p();return{top:t.s.y,left:t.s.x-x.offsetWidth/2}}function l(){var t=p();return{top:t.e.y-x.offsetHeight/2,left:t.e.x}}function u(){var t=p();return{top:t.w.y-x.offsetHeight/2,left:t.w.x-x.offsetWidth}}function f(){var t=p();return{top:t.nw.y-x.offsetHeight,left:t.nw.x-x.offsetWidth}}function i(){var t=p();return{top:t.ne.y-x.offsetHeight,left:t.ne.x}}function a(){var t=p();return{top:t.sw.y,left:t.sw.x-x.offsetWidth}}function c(){var t=p();return{top:t.se.y,left:t.e.x}}function d(){var t=d3.select(document.createElement(\"div\"));return t.style({position:\"absolute\",opacity:0,pointerEvents:\"none\",boxSizing:\"border-box\"}),t.node()}function m(t){return t=t.node(),\"svg\"==t.tagName.toLowerCase()?t:t.ownerSVGElement}function p(){var t=d3.event.target,e={},n=t.getScreenCTM(),r=t.getBBox(),o=r.width,s=r.height,l=r.x,u=r.y,f=document.body.scrollTop,i=document.body.scrollLeft;return document.documentElement&&document.documentElement.scrollTop&&(f=document.documentElement.scrollTop,i=document.documentElement.scrollLeft),w.x=l+i,w.y=u+f,e.nw=w.matrixTransform(n),w.x+=o,e.ne=w.matrixTransform(n),w.y+=s,e.se=w.matrixTransform(n),w.x-=o,e.sw=w.matrixTransform(n),w.y-=s/2,e.w=w.matrixTransform(n),w.x+=o,e.e=w.matrixTransform(n),w.x-=o/2,w.y-=s/2,e.n=w.matrixTransform(n),w.y+=s,e.s=w.matrixTransform(n),e}var y=e,g=n,h=r,x=d(),v=null,w=null;t.show=function(){var e,n=h.apply(this,arguments),r=g.apply(this,arguments),o=y.apply(this,arguments),s=d3.select(x),l=0;for(s.html(n).style({opacity:1,pointerEvents:\"all\"});l--;)s.classed(E[l],!1);return e=T.get(o).apply(this),s.classed(o,!0).style({top:e.top+r[0]+\"px\",left:e.left+r[1]+\"px\"}),t},t.hide=function(){return nodel=d3.select(x),nodel.style({opacity:0,pointerEvents:\"none\"}),t},t.attr=function(e){if(2>arguments.length&&\"string\"==typeof e)return d3.select(x).attr(e);var n=Array.prototype.slice.call(arguments);return d3.selection.prototype.attr.apply(d3.select(x),n),t},t.style=function(e){if(2>arguments.length&&\"string\"==typeof e)return d3.select(x).style(e);var n=Array.prototype.slice.call(arguments);return d3.selection.prototype.style.apply(d3.select(x),n),t},t.direction=function(e){return arguments.length?(y=null==e?e:d3.functor(e),t):y},t.offset=function(e){return arguments.leng

or launch in an external browser like chrome, etc

In [13]:
Open3.popen3("start chrome.exe #{file_path}")

[#<IO:fd 4>, #<IO:fd 5>, #<IO:fd 7>, #<Process::Waiter:0x000000000e89d628 run>]